In [2]:
from dotenv import load_dotenv
import pandas as pd 
import sys, os

load_dotenv()

# pandas settings3
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)



### Datasheet

- /mnt/hdd/octc/PCOS_Dataset/Ovariancyst_datamangerment.csv

In [3]:
filesheet_schema = pd.read_csv('/mnt/hdd/octc/PCOS_Dataset/Ovariancyst_datamangerment.csv')
# 칼럼명 변경
filesheet_schema.columns = ['PIDs', 'USG_Dignosis', 'Pathology_Dignosis', 'label']
filesheet_schema = filesheet_schema.sort_values(by = 'PIDs', ascending= True, ignore_index= True)
print(f"데이터 시트 속 환자 수 : {len(filesheet_schema['PIDs'].unique())}명")

filesheet_schema.info()

데이터 시트 속 환자 수 : 1250명
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1250 entries, 0 to 1249
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   PIDs                1250 non-null   object
 1   USG_Dignosis        1250 non-null   object
 2   Pathology_Dignosis  1250 non-null   object
 3   label               1250 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 39.2+ KB


In [ ]:
filesheet_schema['label'].value_counts()

In [ ]:
# label 칼럼 remapping 1: 양성 / 2: 악성 / 3: 중간형 -> 0: 양성 / 1: 중간형 / 2 : 악성
filesheet_schema_label_fix = filesheet_schema.replace({'label': {1: 0, 2: 2, 3: 1}})
filesheet_schema_label_fix['label'].value_counts()

#### 데이터시트 라벨 표기변경 

In [ ]:
filesheet_schema_label_fix.columns = ['PIDs', 'USG_Dignosis', 'Pathology_Dignosis', 'label|0:양성, 1:중간형, 2:악성']

filesheet_schema_label_fix.to_csv('/mnt/hdd/octc/PCOS_Dataset/라벨표기변경데이터시트.csv', encoding= 'utf-8-sig', index= False)
# filesheet_schema_label_fix

### 제공받은 데이터 :  /mnt/hdd/octc/BACKUP/Dataset

In [ ]:
files = os.listdir(os.getenv('DATA_DIR'))
sorted(files)[:10]

DATA_DIR_Schema = {
    "filenames" : [],
    "PIDs" : [],
    "labels" : [],
    "Counts" : [],
}

for file in files:
    file = file.replace('.png', '')
    
    pid = file.split('_')[1]
    label = file.split('_')[0]
    count = file.split('_')[2]
    
    DATA_DIR_Schema['filenames'].append(file)
    DATA_DIR_Schema['PIDs'].append(pid)
    DATA_DIR_Schema['labels'].append(int(label))
    DATA_DIR_Schema['Counts'].append(count)
    

DATA_DIR_Schema = pd.DataFrame(DATA_DIR_Schema)
DATA_DIR_Schema = DATA_DIR_Schema.sort_values(by = 'PIDs', ascending= True, ignore_index= True)
DATA_DIR_Schema.info()

In [ ]:
DATA_DIR_Schema[DATA_DIR_Schema['Counts'] == '00001']['labels'].value_counts()


In [ ]:
DATA_DIR_Schema.to_csv(os.getenv('DATA_DIR') + '.csv', encoding = 'utf-8-sig', index= False)

#### 같은 PIDs끼리 label 다른지 파악


In [ ]:
# 같은 PIDs끼리 label 다른지 파악
# PIDs별로 유니크한 labels의 개수를 계산합니다.
pid_label_counts = DATA_DIR_Schema.groupby('PIDs')['labels'].nunique()

# labels가 2개 이상인 PIDs를 필터링합니다.
pids_with_diff_labels = pid_label_counts[pid_label_counts > 1].index

print("PIDs가 같은데 labels가 다른 경우의 PIDs:")
print(list(pids_with_diff_labels))

# 해당 PIDs의 데이터를 확인합니다.
result = DATA_DIR_Schema[DATA_DIR_Schema['PIDs'].isin(pids_with_diff_labels)]
print("\n해당 데이터:")
print(result)



# 두 데이터셋에 대한 비교

In [1]:
import pandas as pd

# DATA_DIR_Schema와 filesheet_schema_label_fix 데이터프레임이 이미 있다고 가정합니다.
# filesheet_schema_label_fix의 label 컬럼명이 다소 길어서 간단하게 'label_fix'로 재명명합니다.
filesheet_schema_label_fix = filesheet_schema_label_fix.rename(
    columns={'label|0:양성, 1:중간형, 2:악성': 'label_fix'}
)

# # 중복제거
DATA_DIR_Schema_fix = DATA_DIR_Schema[['PIDs', 'labels']].drop_duplicates()


# 두 DataFrame을 PIDs 기준으로 병합합니다.
merged_df = pd.merge(DATA_DIR_Schema_fix,filesheet_schema_label_fix, on='PIDs', how='inner')
# 같은 PIDs를 가진 행에 대해 두 label이 같은지 비교한 결과를 새로운 컬럼 'label_same'에 저장합니다.
merged_df['label_same'] = merged_df['labels'] == merged_df['label_fix']
merged_df[['PIDs', 'label_fix']].to_csv('./data/label.csv', index= False)

NameError: name 'filesheet_schema_label_fix' is not defined

In [ ]:
# # 결과 확인: label이 다른 경우를 출력
mismatched = merged_df[~merged_df['label_same']]
print(f"같은 PIDs인데 labels가 다른 데이터 개수: {len(mismatched)}")
print(mismatched[['PIDs', 'labels', 'label_fix']])

# Datasheet 정의 : 업데이트된 데이터시트를 토대로 데이터시트 파일을 만듬 

In [21]:
from dotenv import load_dotenv
from lib.seed import seed_prefix 
import sys, os 
load_dotenv()
seed_prefix(seed = 42)

def get_pid(item):
    return item.split('_')[1]


data_dir_path = os.getenv('DATA_DIR')
data_filenames = sorted(os.listdir(data_dir_path))

from lib.dataset import data_split, PCOS_Dataset
import pandas as pd 

label_df = pd.read_csv('/home/eiden/eiden/PCOS-roi-classification/v2/data/라벨표기변경데이터시트.csv')


filenames_to_labels = {
    'filename' : [],
    'label|0:양성, 1:중간형, 2:악성' : []
}
for filename in data_filenames:
    filename = filename.replace('.png', '')
    pid = get_pid(filename)
    label = label_df[label_df['PIDs'] == pid]['label|0:양성, 1:중간형, 2:악성'].values[0]
    
    filenames_to_labels['filename'].append(filename)
    filenames_to_labels['label|0:양성, 1:중간형, 2:악성'].append(label)


datasheet = pd.DataFrame(filenames_to_labels)
datasheet.to_csv('./data/datasheet.csv', index=False)


Seed Fix: 42


### 업데이트 전 데이터시트

In [ ]:
import pandas as pd 
import os 
from dotenv import load_dotenv
load_dotenv()
update_df = pd.read_csv(os.getenv('DATASHEET_PATH'))

before_df = update_df.copy()
for _, row in update_df.iterrows():
    after_label = row['filename'].split('_')[0]
    before_df.iloc[_, 1] = after_label
    
before_df.to_csv('./data/before_datasheet.csv',encoding= 'utf-8-sig', index= False)
before_df

# 라벨 변경 데이터시트에 대한 분석

In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
import pandas as pd 


# dataframe print할때 전체 다 나오게하기 
pd.set_option('display.max_rows', None)


In [2]:
label_sheet = pd.read_csv(os.getenv('MD_LABEL_SHEET_PATH'))

label_sheet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1250 entries, 0 to 1249
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   PIDs                     1250 non-null   object
 1   USG_Dignosis             1250 non-null   object
 2   Pathology_Dignosis       1250 non-null   object
 3   label|0:양성, 1:중간형, 2:악성  1250 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 39.2+ KB


In [20]:
USG_Dignosis = label_sheet['USG_Dignosis']
Pathology_Dignosis = label_sheet['Pathology_Dignosis']

# '.' 이렇게 된 값들은 row 제거
USG_Dignosis = USG_Dignosis.replace('.', 'None')
USG_Dignosis = USG_Dignosis.replace('', 'None')
USG_Dignosis = USG_Dignosis.dropna()
USG_Dignosis_dict= USG_Dignosis.value_counts().to_dict()
USG_Dignosis_schema = pd.DataFrame([USG_Dignosis_dict.keys(), USG_Dignosis_dict.values()]).T
USG_Dignosis_schema.columns = ['USG_Dignosis', 'Counts']
USG_Dignosis_schema[USG_Dignosis_schema['USG_Dignosis'] == ''] = 'None'
USG_Dignosis_schema

,USG_Dignosis,Counts
0,endometrioma,248
1,teratoma,139
2,endometriosis,63
3,malignancy,51
4,benign cyst,51
5,serous cystadenoma,40
6,ovarian cancer,32
7,Endometriotic cyst,29
8,mature cystic teratoma,27
9,borderline to malignancy,23
